In [25]:
import pandas as pd

In [26]:
df = pd.read_parquet("../data/train.parquet")
df

,id,prompt,response_a,response_b,winner,model_a,model_b,language
0,00007cff95d7f7974642a785aca248b0f26e60d3312fac...,vieš po Slovensky?,"Áno, hovorím po slovensky. Ako vám môžem pomôcť?","Áno, veď som tu! Môžem ti pomôcť s otázkami al...",model_a,o1-preview,reka-core-20240904,Slovak
1,00010ed04b536f56ebe43eef1100c13906abea12bf9855...,You will be given a piece of news. Analyze it ...,Let's break down the news and analyze it accor...,"```json\n{\n ""contains_orgs"": true,\n ""orgs""...",model_a,gemma-2-27b-it,gemini-1.5-flash-002,Russian
2,0003800d510e38803efba5ceaec122bc66408fe367b0be...,"Dört basamaklı, rakamları birbirinden ve sıfır...","Bu soruyu çözmek için, verilen koşulları adım ...",Bu problemi adım adım çözelim:\n\n1) ABCD - DC...,model_a,gpt-4-0125-preview,claude-3-5-sonnet-20240620,Turkish
3,00072026c68f5418ef2da238394e418ce72a534b9b22d5...,"현재 추천된 탑 3 종목인 Cabaletta Bio (CABA), Rocket Ph...","죄송하지만 저는 금융 조언을 제공할 수 없습니다. 저는 AI 모델이며, 투자 결정에...",현재 추천된 탑 3 종목에 순위를 매기기 위해서는 여러 가지 요소들을 고려해야 합니...,model_b,gemma-2-2b-it,llama-3.1-nemotron-70b-instruct,English
4,0007ce7cf6bc1b5a8f8a4669b854fb12030863c970d9dc...,Please be boring,"Alright, I'll be as boring as possible.\n\nTod...","Understood. Here is a straightforward, unadorn...",model_a,reka-flash-20240722,grok-2-2024-08-13,English
...,...,...,...,...,...,...,...,...
48434,fff5962462043091a771018e37ece11c425e9c7cca64b2...,你好，请你介绍下你自己,你好！我是LSK，一个形式为人工智能的chatbot。,### 介绍\n\n我是一款人工智能语言模型，基于深度学习技术开发而成。我的主要功能是理解自...,model_b,llama-3.2-3b-instruct,llama-3.1-8b-instruct,Chinese
48435,fffad6c95fb82e3a694b4952965852130285e0a38905d5...,Viva Technology参展费用,Viva Technology是一个国际性的科技展览会，每年在法国巴黎举行。参展费用会根据多...,我无法找到Viva Technology参展费用的具体公开信息。\n\n**参展费用通常取决...,model_a,reka-core-20240904,gemini-1.5-flash-exp-0827,English
48436,fffbf502f2cd88b706a4bd6e54ed90e9a2d59d1643140d...,다음은 '사무장병원 경찰 조사 동행기 - 장효강 변호사의 현장 이야기'라는 제목의 ...,"사무장병원 경찰 조사 동행기 - 장효강 변호사의 현장 이야기\n\n**""압수수색, ...","도입부를 개선하기 위해서는 독자의 관심을 더 끌어야 하며, 좀 더 명료하고 간결하게...",model_a,gemini-1.5-pro-002,yi-lightning-lite,Korean
48437,fffd2ffdaa03e9e0a0cd1e8f2ee80f530bb19b08fa4312...,What is the height of the church located in Sa...,**Church Information: Église de Saint-Nazaire-...,I couldn't find any information about a church...,model_a,llama-3.1-70b-instruct,llama-3.2-3b-instruct,English


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split

# First, let's analyze the combinations
df['stratify_key'] = df['language'] + "_" + df['winner']

# Count samples per combination
combo_counts = df['stratify_key'].value_counts()

# Find rare combinations (count = 1)
rare_combos = combo_counts[combo_counts < 2].index

# Split the data into rare and normal cases
df_rare = df[df['stratify_key'].isin(rare_combos)]
df_normal = df[~df['stratify_key'].isin(rare_combos)]

# For normal cases, do stratified split
if len(df_normal) > 0:
    train, valid = train_test_split(
        df_normal,
        test_size=0.1,
        stratify=df_normal['stratify_key'],
        random_state=42
    )
    
    # Assign splits for normal cases
    df.loc[train.index, 'split'] = 'train'
    df.loc[valid.index, 'split'] = 'valid'

# For rare cases, assign to train by default
df.loc[df_rare.index, 'split'] = 'train'

# Clean up
df.drop(columns=['stratify_key'], inplace=True)

# Print statistics
print("\nRare combinations (count=1):")
print(combo_counts[combo_counts < 2])

print("\nOverall split distribution:")
print(df['split'].value_counts(normalize=True))

print("\nDistribution across language and winner combinations:")
print(df.groupby(['language', 'winner', 'split']).size().unstack(fill_value=0))


Rare combinations (count=1):
stratify_key
Klingon_model_b            1
Nyanja_model_b             1
Volapük_model_a            1
Faroese_model_b            1
Afrikaans_model_b          1
Hawaiian_model_a           1
Tibetan_model_a            1
Icelandic_model_a          1
Welsh_model_b              1
Kurdish_model_a            1
Breton_model_a             1
Lingala_model_b            1
Quechua_model_a            1
Interlingua_model_b        1
Armenian_model_b           1
Turkmen_model_a            1
Kyrgyz_model_b             1
Wolof_model_b              1
Telugu_model_b             1
zzp_model_a                1
Quechua_model_b            1
Volapük_model_b            1
Sindhi_model_b             1
Waray_model_b              1
Latvian_model_b            1
Nauru_model_b              1
Abkhazian_model_a          1
Tsonga_model_a             1
Afar_model_a               1
Inuktitut_model_b          1
Bashkir_model_a            1
Tibetan_model_b            1
Shona_model_a              1


In [28]:
df['split'].value_counts()

split
train    43600
valid     4839
Name: count, dtype: int64

In [29]:
df.to_parquet("../data/train_split.parquet", index=False)

In [30]:
df_loaded = pd.read_parquet("../data/train_split.parquet")
df_loaded

,id,prompt,response_a,response_b,winner,model_a,model_b,language,split
0,00007cff95d7f7974642a785aca248b0f26e60d3312fac...,vieš po Slovensky?,"Áno, hovorím po slovensky. Ako vám môžem pomôcť?","Áno, veď som tu! Môžem ti pomôcť s otázkami al...",model_a,o1-preview,reka-core-20240904,Slovak,train
1,00010ed04b536f56ebe43eef1100c13906abea12bf9855...,You will be given a piece of news. Analyze it ...,Let's break down the news and analyze it accor...,"```json\n{\n ""contains_orgs"": true,\n ""orgs""...",model_a,gemma-2-27b-it,gemini-1.5-flash-002,Russian,valid
2,0003800d510e38803efba5ceaec122bc66408fe367b0be...,"Dört basamaklı, rakamları birbirinden ve sıfır...","Bu soruyu çözmek için, verilen koşulları adım ...",Bu problemi adım adım çözelim:\n\n1) ABCD - DC...,model_a,gpt-4-0125-preview,claude-3-5-sonnet-20240620,Turkish,train
3,00072026c68f5418ef2da238394e418ce72a534b9b22d5...,"현재 추천된 탑 3 종목인 Cabaletta Bio (CABA), Rocket Ph...","죄송하지만 저는 금융 조언을 제공할 수 없습니다. 저는 AI 모델이며, 투자 결정에...",현재 추천된 탑 3 종목에 순위를 매기기 위해서는 여러 가지 요소들을 고려해야 합니...,model_b,gemma-2-2b-it,llama-3.1-nemotron-70b-instruct,English,train
4,0007ce7cf6bc1b5a8f8a4669b854fb12030863c970d9dc...,Please be boring,"Alright, I'll be as boring as possible.\n\nTod...","Understood. Here is a straightforward, unadorn...",model_a,reka-flash-20240722,grok-2-2024-08-13,English,train
...,...,...,...,...,...,...,...,...,...
48434,fff5962462043091a771018e37ece11c425e9c7cca64b2...,你好，请你介绍下你自己,你好！我是LSK，一个形式为人工智能的chatbot。,### 介绍\n\n我是一款人工智能语言模型，基于深度学习技术开发而成。我的主要功能是理解自...,model_b,llama-3.2-3b-instruct,llama-3.1-8b-instruct,Chinese,train
48435,fffad6c95fb82e3a694b4952965852130285e0a38905d5...,Viva Technology参展费用,Viva Technology是一个国际性的科技展览会，每年在法国巴黎举行。参展费用会根据多...,我无法找到Viva Technology参展费用的具体公开信息。\n\n**参展费用通常取决...,model_a,reka-core-20240904,gemini-1.5-flash-exp-0827,English,train
48436,fffbf502f2cd88b706a4bd6e54ed90e9a2d59d1643140d...,다음은 '사무장병원 경찰 조사 동행기 - 장효강 변호사의 현장 이야기'라는 제목의 ...,"사무장병원 경찰 조사 동행기 - 장효강 변호사의 현장 이야기\n\n**""압수수색, ...","도입부를 개선하기 위해서는 독자의 관심을 더 끌어야 하며, 좀 더 명료하고 간결하게...",model_a,gemini-1.5-pro-002,yi-lightning-lite,Korean,train
48437,fffd2ffdaa03e9e0a0cd1e8f2ee80f530bb19b08fa4312...,What is the height of the church located in Sa...,**Church Information: Église de Saint-Nazaire-...,I couldn't find any information about a church...,model_a,llama-3.1-70b-instruct,llama-3.2-3b-instruct,English,valid


In [31]:
df_loaded.split.value_counts()

split
train    43600
valid     4839
Name: count, dtype: int64